In [1]:
## Experiment Option
from easydict import EasyDict
import torch

opt = EasyDict()
opt.dataset_series = 'SemEval-16' # SemEval-16, sentihood
opt.dataset_domain = 'restaurant' # restaurant / laptop
opt.subtask = 'sub1' # sub1: sentence, sub2: document(full review)
opt.task = 'category' # category, term
opt.num_classes = 3 # negative, positive, neutral, (+ conflict)
opt.max_length = 200
opt.model_name = 'bert_attscore_forcls_rnn'
# model_name: {bert_base, bert_attscore, bert_attscore_rnn, bert_attscore_bi_rnn, bert_attscore_rnn_add_asp,
#    bert_attscore_rnn_add_sep1, bert_attscore_rnn_add_sep_both, bert_attscore_forcls_rnn}
opt.pos = False
opt.lastid = False
opt.top_k = 3
opt.valset_ratio = 0.2
opt.batch_size = 16
opt.num_layers = 6
opt.num_epochs = 12
opt.runs = 5
opt.seed = 42
opt.log_step = 100
opt.patience = 5
opt.device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

print(opt.device)

cuda


In [2]:
import os, sys
# research_root = os.path.join(GDRIVE_HOME, 'research')
# sys.path.append(research_root)

if opt.dataset_series == 'SemEval-16':
    path = 'dataset/{}/semeval16_{}_{}.csv'.format(opt.dataset_series, opt.subtask, opt.dataset_domain)
    path_test = 'dataset/{}/semeval16_{}_{}_test.csv'.format(opt.dataset_series, opt.subtask, opt.dataset_domain)
elif opt.dataset_series == 'sentihood':
    path = 'dataset/{}/sentihood_train.csv'.format(opt.dataset_series)
    path_test = 'dataset/{}/sentihood_test.csv'.format(opt.dataset_series)

import pandas as pd

df_train = pd.read_csv(path)
df_test = pd.read_csv(path_test)

print('length of train set: {:,}'.format(len(df_train)))
print('length of test set: {:,}'.format(len(df_test)))

#df_train.head()

length of train set: 2,507
length of test set: 859


In [3]:
if 'attscore' in opt.model_name:
    from data_utils import clean_sentence, preprocess
    df_train = clean_sentence(df=df_train, clean_func=preprocess)
    df_test = clean_sentence(df=df_test, clean_func=preprocess)

## Test analysis
1. Load model
2. Get samples from test set and tokenize
3. Predict
4. Decode high attention words
5. Result DataFrame

#### Load Model

In [4]:
opt.model_name

'bert_attscore_forcls_rnn'

In [5]:
from models.bert_intermediate import *
from models.bert_pos import *
from models.bert_attscores import *

if opt.model_name == 'bert_base':
    model = Bert_Base(opt.num_classes)
elif opt.model_name == 'bert_attscore':
    model = Bert_AttScore(opt=opt, embed_dim=768, fc_hid_dim=128, top_k=3, att_head='all', att_pooling='mean')
elif opt.model_name == 'bert_attscore_rnn':
    model = Bert_AttScore_RNN(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=False,
                              top_k=opt.top_k, att_head='all', att_pooling='gru')
elif opt.model_name == 'bert_attscore_bi_rnn':
    model = Bert_AttScore_RNN(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=True,
                              top_k=opt.top_k, att_head='all', att_pooling='gru')
elif opt.model_name == 'bert_attscore_rnn_add_sep1':
    model = Bert_AttScore_RNN_add(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=True,
                                 top_k=opt.top_k, att_head='all', additional_token='sep1', att_pooling='gru')
elif opt.model_name == 'bert_attscore_rnn_add_sep2':
    model = Bert_AttScore_RNN_add(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=True,
                                 top_k=opt.top_k, att_head='all', additional_token='sep2', att_pooling='gru')
elif opt.model_name == 'bert_attscore_rnn_add_sep_both':
    model = Bert_AttScore_RNN_add(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=True,
                                 top_k=opt.top_k, att_head='all', additional_token='sep_both', att_pooling='gru')
elif opt.model_name == 'bert_attscore_rnn_add_asp':
    model = Bert_AttScore_RNN_add(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=True,
                                 top_k=opt.top_k, att_head='all', additional_token='asp', att_pooling='gru')
elif opt.model_name == 'bert_attscore_rnn_add_cls':
    model = Bert_AttScore_RNN_add(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=True,
                                 top_k=opt.top_k, att_head='all', additional_token='cls', att_pooling='gru')
elif opt.model_name == 'bert_attscore_forcls_rnn':
    model = Bert_AttScore_forCLS_RNN(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=True,
                                 top_k=opt.top_k, att_head='all', att_pooling='gru')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model.load_state_dict(torch.load('state_dict/BEST_bert_attscore_forcls_rnn_SemEval-16_restaurant_val_acc_81.64%'))

<All keys matched successfully>

#### Get samples from test set and tokenize

In [7]:
df_samples = df_test.sample(30).reset_index(drop=True)
from data_utils import Category_Classification_Dataset as Dataset
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
#trainset = Dataset(df=df_train, tokenizer=tokenizer, opt=opt, pos_encoding=False)
samples = Dataset(df=df_samples, tokenizer=tokenizer, opt=opt, pos_encoding=False)

from torch.utils.data import DataLoader
sample_loader = DataLoader(dataset=samples, batch_size=30, shuffle=False)
iter_sample = iter(sample_loader).next()
print(len(iter_sample['input_ids']))

sentihood: False
30 samples in this dataset
30


#### Predict

In [8]:
output, top_k_idx = model(iter_sample['input_ids'].squeeze(1), iter_sample['attention_masks'].squeeze(1),
                         iter_sample['token_type_ids'].squeeze(1))
predicted = torch.argmax(output, axis=1)
labels = iter_sample['labels']

correct = (predicted == labels).tolist()
labels = labels.tolist()
predicted = predicted.tolist()

#### Decode high attention words

In [9]:
high_tokens = list()
for tokens, idx in zip(iter_sample['input_ids'].squeeze(1), top_k_idx):
    high_tokens.append(tokens.squeeze(0)[idx])

words = [tokenizer.decode(tokens) for tokens in high_tokens]

#### Result DataFrame

In [10]:
map_result = {0: 'negative', 1: 'positive', 2: 'neutral'}
predict = [map_result[i] for i in predicted]
truth = [map_result[i] for i in labels]

result_dict = {'sentence': df_samples['sentence'], 'aspect': df_samples['category'],
              'high_atts': words, 'truth': truth, 'predict': predict, 'correct': correct}

df = pd.DataFrame(result_dict)
df

,sentence,aspect,high_atts,truth,predict,correct
0,plus i am allergic to rice and the waitstaff w...,SERVICE#GENERAL,was - ',positive,positive,True
1,Love their pizza especially the mushroom pizza,FOOD#QUALITY,love their mushroom,positive,positive,True
2,The chicken curry and chicken tikka masala are...,FOOD#QUALITY,are my favorite,positive,positive,True
3,Everything you want and more very fresh,FOOD#STYLE_OPTIONS,everything more very,positive,positive,True
4,Food wise its ok but a bit pricey for what you...,AMBIENCE#GENERAL,wise its but,neutral,neutral,True
5,Freshest sushi I love this restaurant,FOOD#QUALITY,i love this,positive,positive,True
6,Fresh ingrediants and super tasty,FOOD#QUALITY,fresh and super,positive,positive,True
7,My wife also ordered a teapot of hot water (sh...,SERVICE#GENERAL,also ( ',negative,negative,True
8,The waiter was a bit unfriendly and the feel o...,AMBIENCE#GENERAL,and restaurant crowded,negative,negative,True
9,Never had one single problem there,RESTAURANT#GENERAL,never problem there,positive,positive,True


#### Samples

In [11]:
def check_att(result_df, idx):
    print('Sentence: ', result_df.iloc[idx].sentence)
    print('Aspect: ', result_df.iloc[idx].aspect)
    print('High_att_words: ', result_df.iloc[idx].high_atts)
    print('Predicted: {} | Truth: {}'.format(result_df.iloc[idx].predict, result_df.iloc[idx].truth))

In [29]:
check_att(df, 17)

Sentence:   Great drinks  corn beef hash coffee B Fast burritos Gluten Free menu
Aspect:  FOOD#STYLE_OPTIONS
High_att_words:  great corn b
Predicted: positive | Truth: positive
